In [2]:
import sys
import os

# --- CRITICAL SETUP ---
# We point directly to the project root so we can import 'config' and 'spark_jobs'
PROJECT_ROOT = "/home/talentum/Distributed-log-analyzer"
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

# Now we can import your custom modules
from spark_jobs.common.spark_utils import get_spark_session, load_config
import config.schemas as schemas
from pyspark.sql.functions import input_file_name, current_timestamp, regexp_extract, col

print("Setup Complete. Project modules loaded.")

ModuleNotFoundError: No module named 'pyspark'